In [63]:
from gurobipy import Model, GRB
import polars as pl  # Using Polars for educational purposes
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt

stocks_1 = pl.read_csv("stocks.csv")

In [64]:
stocks_1.head()

Date,AAPL,GOOG,IBM,MARA,NVDA
str,f64,f64,f64,f64,f64
"""1/16/2023""",137.87,99.28,141.2,8.07,178.39
"""1/23/2023""",145.118851,100.709999,134.389999,8.02,203.649994
"""1/30/2023""",153.641205,105.220001,136.940002,7.07,211.0
"""2/6/2023""",150.170624,94.860001,135.600006,5.92,212.649994
"""2/13/2023""",151.933685,94.589996,135.020004,7.7,213.880005


In [65]:
# Get all the stock names to iterate over
stock_names = stocks_1.columns[1:]

# Calculate the weekly return percentage for each stock
stocks_1 = (
    stocks_1.with_columns(
        ((pl.col(s) - pl.col(s).shift(1)) / pl.col(s).shift(1) * 100.0).alias(s)
        for s in stock_names
    )
    .filter(pl.col("Date") != "1/16/2023")
    .drop("Date")
)
stocks_1

AAPL,GOOG,IBM,MARA,NVDA
f64,f64,f64,f64,f64
5.257744,1.44037,-4.822947,-0.619579,14.159983
5.872672,4.478207,1.897465,-11.845387,3.609136
-2.258887,-9.846037,-0.978528,-16.265912,0.781988
1.174039,-0.284635,-0.42773,30.067568,0.57842
-3.828253,-5.539696,-3.295806,-13.376623,8.874133
2.944577,5.226636,-0.712268,-4.347826,2.59383
-1.675162,-3.201441,-3.232029,-16.45768,-3.871913
4.377104,12.581031,-1.402945,53.283302,12.018292
3.387098,3.513565,1.293556,-4.406365,4.097185


In [66]:
def calculate_covariance(df: pl.DataFrame):
    # Center columns with the mean
    centered_df = df.select(
        [(pl.col(c) - pl.col(c).mean()).alias(c) for c in df.columns]
    )

    # Calculate pairwise covariance and construct covariance matrix
    n = len(df) - 1
    cov_matrix = pl.DataFrame(
        [
            [(centered_df[c1] * centered_df[c2]).sum() / n for c1 in df.columns]
            for c2 in df.columns
        ],
        schema=df.schema,
    )

    return cov_matrix


n_stocks = len(stock_names)
stock_return = stocks_1.mean()
cov_mat = calculate_covariance(stocks_1)
cov_mat

AAPL,GOOG,IBM,MARA,NVDA
f64,f64,f64,f64,f64
8.891972,6.163829,2.499224,9.3736,9.096995
6.163829,19.498635,1.234442,17.023391,9.833599
2.499224,1.234442,6.123906,-1.107004,2.074088
9.3736,17.023391,-1.107004,292.463474,1.309068
9.096995,9.833599,2.074088,1.309068,36.943424


In [67]:
# Verifying covariance matrix calculation with Pandas calculation
stocks_1.to_pandas().cov()

,AAPL,GOOG,IBM,MARA,NVDA
AAPL,8.891972,6.163829,2.499224,9.373600,9.096995
GOOG,6.163829,19.498635,1.234442,17.023391,9.833599
IBM,2.499224,1.234442,6.123906,-1.107004,2.074088
MARA,9.373600,17.023391,-1.107004,292.463474,1.309068
NVDA,9.096995,9.833599,2.074088,1.309068,36.943424


# Optimization Model

In [68]:
cov_mat = cov_mat.to_pandas()
cov_mat.index = stock_names

stock_return = stock_return.to_pandas().loc[0]

m = Model("Portfolio")

vars = pd.Series(m.addVars(stock_names, lb=0), index=stock_names)

portfolio_risk = cov_mat.dot(vars).dot(vars)
m.setObjective(portfolio_risk, GRB.MINIMIZE)

m.addConstr(vars.sum() == 1, "budget")
m.addConstr(stock_return.dot(vars) >= 0.009, "return")

m.optimize()

for v in vars:
    print(f"{v.varName}: {v.x}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D56)

CPU model: Apple M2 Pro
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2 rows, 5 columns and 10 nonzeros
Model fingerprint: 0x7db700ff
Model has 15 quadratic objective terms
Coefficient statistics:
  Matrix range     [4e-01, 3e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [4e+00, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e-03, 1e+00]
Presolve time: 0.00s
Presolved: 2 rows, 5 columns, 10 nonzeros
Presolved model has 15 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 4
 AA' NZ     : 1.500e+01
 Factor NZ  : 2.100e+01
 Factor Ops : 9.100e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   5.72937239e+07 -5.72937239e+07  4.00e+03 3.51e-06  1.00e+06     0

# Efficient Frontier

In [83]:
stocks_2 = pl.read_csv("stocks.csv")
stock_names = stocks_2.columns[1:]

stocks_2 = (
    stocks_2.with_columns(
        *[
            ((pl.col(s) - pl.col(s).shift(1)) / pl.col(s).shift(1) * 100.0).alias(s)
            for s in stock_names
        ],
        pl.col("Date").str.to_date("%m/%d/%Y").alias("Date"),
    )
    .with_columns(
        [
            pl.col("Date").dt.year().alias("year"),
            pl.col("Date").dt.month().alias("month"),
        ]
    )
    .filter(pl.col("Date") != pl.date(2023, 1, 16))
    .group_by(["year", "month"])
    .agg(pl.all().sum())
    .drop("Date", "year", "month")
)
stocks_2

AAPL,GOOG,IBM,MARA,NVDA
f64,f64,f64,f64,f64
8.990764,10.950861,1.287838,44.070986,15.970357
6.658355,15.29672,4.779337,1.713685,38.036822
-6.07606,6.849234,7.656165,15.513621,5.983529
-9.906747,-3.546322,-5.189286,-32.748882,-10.400095
3.412034,-0.325277,5.463366,11.511057,4.563411
0.725659,5.821642,1.859081,65.461232,5.792273
2.89736,4.163676,-3.598409,28.882894,-0.02377
11.130415,5.918576,-2.925482,-12.464965,17.76912
8.234623,2.368349,8.312548,40.353785,4.188053


In [85]:
stock_volatility = stocks_2.std()
stock_return = stocks_2.mean()

AAPL,GOOG,IBM,MARA,NVDA
f64,f64,f64,f64,f64
6.074022,6.680909,4.770849,31.281188,11.551896
